# Lesson 2 : LangGraph Components

In [ ]:
from dotenv import load_dotenv
_ = load_dotenv()

In [ ]:
from langgraph.graph import StateGraph, END

# Typings are used to create the agent state.
from typing import TypedDict, Annotated
import operator

# Different LangChain message types that are used to represent the human, AI, and system messages.
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

# LangChain wrapper around the OpenAI API. This exposes a standard interface for all language models.
# This means that, even though we're using OpenAI to chat with, we can switch it out for any LangChain supported models.
from langchain_openai import ChatOpenAI

# Search results.
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

> If you are not familiar with python typing annotation, you can refer to the [python documents](https://docs.python.org/3/library/typing.html).

In [ ]:
# The agent state is just an annotated list of messages that we'll add to over time.
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

> Note: in `take_action` below, some logic was added to cover the case that the LLM returned a non-existent tool name. Even with function calling, LLMs can still occasionally hallucinate. Note that all that is done is instructing the LLM to try again! An advantage of an agentic organization.

In [ ]:
# We'll need three functions: 
# One function to call OpenAI
# One function to check whether there is an action present.
# One function to take that action.

class Agent:

    def __init__(self, model, tools, system=""):
        # Save system message.
        self.system = system

        graph = StateGraph(AgentState)

        # Node that executes the LLM.
        graph.add_node("llm", self.call_openai)

        # Action node.
        graph.add_node("action", self.take_action)

        # Conditional action. Checks if there's an action present. Proceeds to the action if it's present. Finishes if there isn't.
        # llm --> exists_action
        #   if exists_action == True --> action
        #   if exists_action == False --> END
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END} # Dictionary to determine how to map the nodes.
        )

        # Add a regular edge from the action node to the LLM node.
        # action --> llm
        graph.add_edge("action", "llm")

        # Indicate where the graph should be entered.
        graph.set_entry_point("llm")

        # Compiles the graph after all the setup to create a LangChain runnable.
        # A LangChain runnable exposes a standard interface for calling and invoking this graph.
        # This will be saved as an attribute on the graph.
        self.graph = graph.compile()

        # We'll save the tools we used for this graph.
        self.tools = {t.name: t for t in tools}

        # Binds the tools we can use to the model to let the model know it has these tools.
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost

# Pass in the model.
# Pass in the tools used. (We're only using one, the TavilySearchResults.)
# Pass in the system prompt.
abot = Agent(model, [tool], system=prompt)

In [ ]:
# We can visulize this graph we created.
from IPython.display import Image

Image(abot.graph.get_graph().draw_png())

In [ ]:
# Create a human message representing a user message.
# We have to do this because the state that the agent expects to work with has this message attribute, which is a list of messages.
# We need to make it conform to this state.
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
# The final state that the agent ends at.
result

In [ ]:
# The answer/final message.
result['messages'][-1].content

In [ ]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
result['messages'][-1].content

In [ ]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

In [ ]:
print(result['messages'][-1].content)